In [15]:
# for CF
import pandas as pd
import numpy as np
import time
from sklearn import cross_validation as cv
from sklearn import preprocessing
from operator import itemgetter
from collections import OrderedDict
from sklearn.preprocessing import StandardScaler
from numpy import *

In [8]:
# read data
file_name = 'ratings.data'
with open(file_name,'r') as f:
    data = [x.strip().split('\t') for x in f]
user_id = [item[0] for item in data]
item_id = [item[1] for item in data]
rating = [item[2] for item in data]
time_stamp = [item[3] for item in data]

uir_df = pd.DataFrame(
    {'user_id': user_id,
     'item_id': item_id,
     'rating': rating
    })

In [7]:
# user_item matrix generation
# row: user
# col: item
def u_i_mat_generate(df, all_user_id, all_item_id):
    find = lambda searchList, elem: [[i for i, x in enumerate(searchList) if x == e] for e in elem]
    user_item_mat = []
    
    for i in range(len(all_user_id)):
        temp_rating_list = np.array([0] * len(all_item_id))
        pos = find(all_item_id, df.loc[df['user_id'] == all_user_id[i]]['item_id'].values.tolist())
        pos = [y for x in pos for y in x]
        temp_rating_list[pos] = df.loc[df['user_id'] == all_user_id[i]]['rating'].values.tolist()
        user_item_mat.append(temp_rating_list)
    user_item_mat = np.matrix(user_item_mat)
    
    return user_item_mat

In [ ]:
def pred_out()

In [17]:
# CF predict function
def CF_pred(k, user_item, cos_pcc, training_data, testing_data):
    if user_item != 'user' and user_item != 'item':
        return print('Check your user_item!')
    if cos_pcc != 'cos' and cos_pcc != 'pcc':
        return print('Check your cos_pcc')
# ------
# ------
    all_user_id = pd.unique(training_data['user_id']).tolist()
    all_item_id = pd.unique(training_data['item_id']).tolist()
    test_user_id = testing_data['user_id'].tolist()
    test_item_id = testing_data['item_id'].tolist()
    
    start_time = time.time()
    user_item_mat = u_i_mat_generate(training_data, all_user_id, all_item_id)
    print("--- %s mat generate seconds ---" % (time.time() - start_time))
    start_time = time.time()
    
    if user_item == 'user':
        if cos_pcc == 'cos':
            mat_normalized = preprocessing.normalize(user_item_mat, norm='l2', axis = 1)
            mat_normalized = mat_normalized.dot(mat_normalized.T)
        elif cos_pcc == 'pcc':
            print('here 3')
            mat_normalized = user_item_mat - mean(user_item_mat, axis = 1)
            mat_normalized = preprocessing.normalize(mat_normalized, norm='l2', axis = 1)
            mat_normalized = mat_normalized.dot(mat_normalized.T)
            print("--- %s pcc mat seconds ---" % (time.time() - start_time))

    elif user_item == 'item':
        if cos_pcc == 'cos':
            mat_normalized = preprocessing.normalize(user_item_mat , norm='l2', axis = 0)
            mat_normalized = mat_normalized.T.dot(mat_normalized)
        elif cos_pcc == 'pcc':
            mat_normalized = mat_normalized - mean(mat_normalized, axis = 0)
            mat_normalized = preprocessing.normalize(mat_normalized, norm='l2', axis = 0)
            mat_normalized = mat_normalized.T.dot(mat_normalized)
            
    return mat_normalized
    

In [9]:
train_data, test_data = cv.train_test_split(uir_df, test_size=0.25)

In [18]:
a = CF_pred(100, 'user', 'pcc', train_data, test_data)

--- 41.87316274642944 mat generate seconds ---
here 3
--- 0.06247115135192871 pcc mat seconds ---


In [43]:
b = np.mat()

array([[ 1.00000000e+00,  2.24582424e-01,  1.98855922e-01, ...,
         6.30611962e-02,  1.72471711e-03, -1.11555619e-02],
       [ 2.24582424e-01,  1.00000000e+00,  8.74771267e-02, ...,
         9.77582969e-02,  1.82040174e-01,  4.25857456e-02],
       [ 1.98855922e-01,  8.74771267e-02,  1.00000000e+00, ...,
         9.34840241e-04,  1.16800814e-02, -1.73678172e-02],
       ...,
       [ 6.30611962e-02,  9.77582969e-02,  9.34840241e-04, ...,
         1.00000000e+00,  4.05557593e-02, -7.61220559e-03],
       [ 1.72471711e-03,  1.82040174e-01,  1.16800814e-02, ...,
         4.05557593e-02,  1.00000000e+00,  1.08308874e-01],
       [-1.11555619e-02,  4.25857456e-02, -1.73678172e-02, ...,
        -7.61220559e-03,  1.08308874e-01,  1.00000000e+00]])